In [ ]:
import os
# change the OS to use your project folder as the working directory
project_folder = "/content/drive/MyDrive/Colab_Notebooks/depression_detection/"
os.chdir(project_folder)

In [ ]:
import pandas as pd
final_text=[]
for file in os.listdir(project_folder):
  if file.endswith("_TRANSCRIPT.csv"):
    data_features = pd.read_csv(file,delimiter='\t')
    data_values= data_features['value']
    value_list = data_values.tolist()
    final_text.append(value_list)

In [ ]:
import pandas as pd

train_labels = pd.read_csv('train_split_Depression_AVEC2017.csv')
dev_labels = pd.read_csv('dev_split_Depression_AVEC2017.csv')
test_labels = pd.read_csv('full_test_split.csv')


In [ ]:
test_labels=test_labels.rename(columns={"PHQ_Binary": "PHQ8_Binary"})

In [ ]:
# If not, please adjust the column names accordingly
combined_data = pd.concat([train_labels, dev_labels])
combined_data = pd.concat([combined_data, test_labels])

In [ ]:
combined_data=combined_data.sort_values(by=['Participant_ID'])

### **CHATGPT 4**


In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import cross_val_score, cross_validate
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import make_pipeline
from sklearn.metrics import make_scorer, recall_score, f1_score, confusion_matrix


# Extract the 'PHQ8_Binary' label
labels = combined_data['PHQ8_Binary'].tolist()

import numpy as np
import pandas as pd
from sklearn.model_selection import cross_validate
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import make_scorer, recall_score, confusion_matrix

# Custom scorer for specificity
def specificity_score(y_true, y_pred):
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
    return tn / (tn + fp)


# Flatten each inner list to a single string
flattened_text = [' '.join(map(str, doc)) for doc in final_text]


# Define the pipeline
pipeline = make_pipeline(
    CountVectorizer(),  # Convert text data to numeric features
    LogisticRegression(max_iter=1000)  # Apply a logistic regression model
)

# Define scorers
scoring = {
    'f1_score': 'f1',  # Directly use 'f1' string shortcut for f1 score
    'sensitivity': make_scorer(recall_score),  # recall is the same as sensitivity
    'specificity': make_scorer(specificity_score)
}

# Perform 5-fold cross-validation using the pipeline and text data
results = cross_validate(pipeline, flattened_text, labels, cv=5, scoring=scoring, return_train_score=False)

# Calculate mean and standard deviation for F1, sensitivity, and specificity
f1_mean = np.mean(results['test_f1_score'])
f1_std = np.std(results['test_f1_score'])
sensitivity_mean = np.mean(results['test_sensitivity'])
sensitivity_std = np.std(results['test_sensitivity'])
specificity_mean = np.mean(results['test_specificity'])
specificity_std = np.std(results['test_specificity'])

# Print results
print("F1 Score - Mean: {:.3f}, Standard Deviation: {:.3f}".format(f1_mean, f1_std))
print("Sensitivity - Mean: {:.3f}, Standard Deviation: {:.3f}".format(sensitivity_mean, sensitivity_std))
print("Specificity - Mean: {:.3f}, Standard Deviation: {:.3f}".format(specificity_mean, specificity_std))


F1 Score - Mean: 0.251, Standard Deviation: 0.101
Sensitivity - Mean: 0.253, Standard Deviation: 0.135
Specificity - Mean: 0.731, Standard Deviation: 0.083


In [ ]:
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, f1_score, recall_score, confusion_matrix
from nltk.tokenize import word_tokenize
import nltk
nltk.download('punkt')


# Symptom keywords and their associated weights

# Mapping of checklist items to keywords
symptom_keywords = {
    "sad": 3, "depressed": 3, "down": 2,
    "unhappy": 2, "blue": 2, "miserable": 2,
    "crying": 2, "tearful": 2, "weeping": 2,
    "discouraged": 2, "disheartened": 2, "hopeless": 3,
    "worthless": 3, "inadequate": 3, "inferior": 2,
    "guilty": 2, "ashamed": 2, "remorse": 1,
    "self-criticism": 1, "blaming": 1,
    "indecisive": 1, "hesitant": 1, "uncertain": 1,
    "disinterest in relationships": 2, "social withdrawal": 2, "isolation": 2,
    "lonely": 2, "isolated": 2, "alone": 2,
    "avoiding social interactions": 2, "withdrawal": 2, "seclusion": 2,
    "unmotivated": 2, "apathetic": 2, "listless": 2,
    "disinterest in work": 2, "bored": 2, "apathetic about activities": 2,
    "avoiding responsibilities": 2, "neglecting duties": 2, "shirking": 1,
    "anhedonia": 3, "joyless": 3, "pleasureless": 3,
    "fatigue": 1, "exhausted": 1, "worn out": 1,
    "insomnia": 1, "oversleeping": 1, "sleep disturbances": 1,
    "eating less": 1, "eating more": 1, "appetite changes": 1,
    "low libido": 1, "disinterest in sex": 1, "sexual apathy": 1,
    "health anxiety": 1, "hypochondria": 1, "preoccupied with health": 1
}


# Function to analyze text and generate a rating based on symptom keywords
def generate_rating(text):
    words = word_tokenize(text.lower())
    return sum(symptom_keywords.get(word, 0) for word in words)

# Flatten each inner list to a single string
flattened_text = [' '.join(map(str, doc)) for doc in final_text]
ratings = [generate_rating(text) for text in flattened_text]

# Convert text data to numeric features using CountVectorizer within a pipeline
vectorizer = CountVectorizer()
X_text = vectorizer.fit_transform(flattened_text).toarray()

# Append ratings to feature matrix
ratings_array = np.array(ratings).reshape(-1, 1)
X_combined = np.hstack((X_text, ratings_array))

# Extract the 'PHQ8_Binary' label
labels = combined_data['PHQ8_Binary'].tolist()


# Define the pipeline with LogisticRegression
pipeline = make_pipeline(
    LogisticRegression(max_iter=1000)
)

# Define custom scorers
def specificity_score(y_true, y_pred):
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
    return tn / (tn + fp)

scoring = {
    'f1_score': 'f1',
    'sensitivity': make_scorer(recall_score),
    'specificity': make_scorer(specificity_score)
}

# Perform 5-fold cross-validation using the pipeline and combined data
results = cross_validate(pipeline, X_combined, labels, cv=5, scoring=scoring, return_train_score=False,  error_score='raise')

# Calculate and print the mean and standard deviation for each score
f1_mean = np.mean(results['test_f1_score'])
f1_std = np.std(results['test_f1_score'])
sensitivity_mean = np.mean(results['test_sensitivity'])
sensitivity_std = np.std(results['test_sensitivity'])
specificity_mean = np.mean(results['test_specificity'])
specificity_std = np.std(results['test_specificity'])

print("F1 Score - Mean: {:.3f}, Standard Deviation: {:.3f}".format(f1_mean, f1_std))
print("Sensitivity - Mean: {:.3f}, Standard Deviation: {:.3f}".format(sensitivity_mean, sensitivity_std))
print("Specificity - Mean: {:.3f}, Standard Deviation: {:.3f}".format(specificity_mean, specificity_std))

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


F1 Score - Mean: 0.245, Standard Deviation: 0.101
Sensitivity - Mean: 0.253, Standard Deviation: 0.135
Specificity - Mean: 0.708, Standard Deviation: 0.068


## **chatgpt 4o**

In [ ]:
import pandas as pd
from sklearn.model_selection import KFold
from sklearn.metrics import f1_score, recall_score, confusion_matrix, precision_score
from sklearn.feature_extraction.text import TfidfVectorizer
from imblearn.over_sampling import SMOTE
import numpy as np
import xgboost as xgb

# Define function to calculate specificity for binary classification
def specificity_score(y_true, y_pred):
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
    return tn / (tn + fp) if (tn + fp) != 0 else 0

# Example data
# final_text should be your processed text data
# labels should be your binary labels

# Assuming final_text is a list of lists containing tokens and labels is a list of binary labels

# Flatten each inner list to a single string
flattened_text = [' '.join(map(str, doc)) for doc in final_text]

# Vectorize the text data with bigrams included
vectorizer = TfidfVectorizer(ngram_range=(1, 2))
X = vectorizer.fit_transform(flattened_text)
y = np.array(labels)  # Ensure y is a numpy array

kf = KFold(n_splits=5)

f1_scores = []
sensitivities = []
specificities = []
precisions = []

for train_index, test_index in kf.split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

    # Handle class imbalance using SMOTE
    smote = SMOTE()
    X_train, y_train = smote.fit_resample(X_train, y_train)

    model = xgb.XGBClassifier(use_label_encoder=False, eval_metric='logloss')
    model.fit(X_train, y_train)

    y_pred = model.predict(X_test)

    f1 = f1_score(y_test, y_pred)
    sensitivity = recall_score(y_test, y_pred)
    specificity = specificity_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)

    f1_scores.append(f1)
    sensitivities.append(sensitivity)
    specificities.append(specificity)
    precisions.append(precision)

f1_mean = np.mean(f1_scores)
f1_std = np.std(f1_scores)
sensitivity_mean = np.mean(sensitivities)
sensitivity_std = np.std(sensitivities)
specificity_mean = np.mean(specificities)
specificity_std = np.std(specificities)
precision_mean = np.mean(precisions)
precision_std = np.std(precisions)

print(f"F1 Score: Mean = {f1_mean}, Std Dev = {f1_std}")
print(f"Sensitivity: Mean = {sensitivity_mean}, Std Dev = {sensitivity_std}")
print(f"Specificity: Mean = {specificity_mean}, Std Dev = {specificity_std}")
print(f"Precision: Mean = {precision_mean}, Std Dev = {precision_std}")


F1 Score: Mean = 0.1407719298245614, Std Dev = 0.07419837048191977
Sensitivity: Mean = 0.10935508935508938, Std Dev = 0.06209587984072962
Specificity: Mean = 0.8242917887461498, Std Dev = 0.06271873115721181
Precision: Mean = 0.22666666666666666, Std Dev = 0.13727506854649335


In [ ]:
import pandas as pd
from sklearn.model_selection import KFold
from sklearn.metrics import f1_score, recall_score, confusion_matrix, precision_score
from sklearn.feature_extraction.text import TfidfVectorizer
from imblearn.over_sampling import SMOTE
import numpy as np
import xgboost as xgb

# Define function to calculate specificity for binary classification
def specificity_score(y_true, y_pred):
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
    return tn / (tn + fp) if (tn + fp) != 0 else 0

# Example data
# final_text should be your processed text data
# labels should be your binary labels

# Assuming final_text is a list of lists containing tokens and labels is a list of binary labels

# Flatten each inner list to a single string
flattened_text = [' '.join(map(str, doc)) for doc in final_text]

# Vectorize the text data with bigrams included
vectorizer = TfidfVectorizer(ngram_range=(1, 2))
X = vectorizer.fit_transform(flattened_text)
y = np.array(labels)  # Ensure y is a numpy array

kf = KFold(n_splits=5)

f1_scores = []
sensitivities = []
specificities = []
precisions = []

misclassified_data = []

for train_index, test_index in kf.split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

    # Handle class imbalance using SMOTE
    smote = SMOTE()
    X_train, y_train = smote.fit_resample(X_train, y_train)

    model = xgb.XGBClassifier(use_label_encoder=False, eval_metric='logloss')
    model.fit(X_train, y_train)

    y_pred = model.predict(X_test)

    # Identify misclassified points and store the original text
    misclassified_indices = test_index[y_test != y_pred]
    misclassified_data.extend([(i, flattened_text[i], y_test[j], y_pred[j]) for j, i in enumerate(misclassified_indices) if y_test[j] != y_pred[j]])

    f1 = f1_score(y_test, y_pred)
    sensitivity = recall_score(y_test, y_pred)
    specificity = specificity_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)

    f1_scores.append(f1)
    sensitivities.append(sensitivity)
    specificities.append(specificity)
    precisions.append(precision)

f1_mean = np.mean(f1_scores)
f1_std = np.std(f1_scores)
sensitivity_mean = np.mean(sensitivities)
sensitivity_std = np.std(sensitivities)
specificity_mean = np.mean(specificities)
specificity_std = np.std(specificities)
precision_mean = np.mean(precisions)
precision_std = np.std(precisions)

print(f"F1 Score: Mean = {f1_mean}, Std Dev = {f1_std}")
print(f"Sensitivity: Mean = {sensitivity_mean}, Std Dev = {sensitivity_std}")
print(f"Specificity: Mean = {specificity_mean}, Std Dev = {specificity_std}")
print(f"Precision: Mean = {precision_mean}, Std Dev = {precision_std}")

# Print misclassified data points
print("\nMisclassified Data Points (index, original_text, true_label, predicted_label):")
for data in misclassified_data:
    print(data)


F1 Score: Mean = 0.1934199134199134, Std Dev = 0.14079913385573778
Sensitivity: Mean = 0.22864801864801865, Std Dev = 0.23815695103382206
Specificity: Mean = 0.8667472015626174, Std Dev = 0.09808370621996375
Precision: Mean = 0.33636363636363636, Std Dev = 0.2616697811749864

Misclassified Data Points (index, original_text, true_label, predicted_label):
(38, "hi i'm ellie thanks for coming in today i was created to talk to people in a safe and secure environment i'm not a therapist but i'm here to learn about people and would love to learn about you i'll ask a few questions to get us started and please feel free to tell me anything your answers are totally confidential  are you okay with this yeah okay so how are you doing today i'm okay that's good where are you from originally um chicago illinois really mhm why'd you move to l_a uh my husband's work was here when'd you move to l_a in eighty four how easy was it for you to get used to living in l_a it was not easy for me it took about

In [ ]:
import pandas as pd
from sklearn.model_selection import KFold
from sklearn.metrics import f1_score, recall_score, confusion_matrix, precision_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LinearRegression
from imblearn.over_sampling import SMOTE
import numpy as np
import xgboost as xgb

# Define function to calculate specificity for binary classification
def specificity_score(y_true, y_pred):
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
    return tn / (tn + fp) if (tn + fp) != 0 else 0

# Example data
# final_text should be your processed text data
# labels should be your binary labels

# Assuming final_text is a list of lists containing tokens and labels is a list of binary labels

# Flatten each inner list to a single string
flattened_text = [' '.join(map(str, doc)) for doc in final_text]

# Vectorize the text data with bigrams included
vectorizer = TfidfVectorizer(ngram_range=(1, 2))
X_text = vectorizer.fit_transform(flattened_text)

# Define the symptoms from Burn’s Depression Checklist
symptoms = [
    'Feeling sad or down in the dumps', 'Feeling unhappy or blue', 'Crying spells or tearfulness',
    'Feeling discouraged', 'Feeling hopeless', 'Low self-esteem', 'Feeling worthless or inadequate',
    'Guilt or shame', 'Criticizing yourself or blaming others', 'Difficulty making decisions',
    'Loss of interest in family, friends or colleagues', 'Loneliness', 'Spending less time with family or friends',
    'Loss of motivation', 'Loss of interest in work or other activities', 'Avoiding work or other activities',
    'Loss of pleasure or satisfaction in life', 'Feeling tired', 'Difficulty sleeping or sleeping too much',
    'Decreased or increased appetite', 'Loss of interest in sex', 'Worrying about your health',
    'Do you have any suicidal thoughts?', 'Would you like to end your life?', 'Do you have a plan for harming yourself?'
]

# Placeholder for symptom ratings
symptom_ratings = np.zeros((X_text.shape[0], len(symptoms)))

# Regression model to predict symptom ratings (for simplicity, using Linear Regression)
regressor = LinearRegression()

# Train a regression model for each symptom and predict ratings
for i, symptom in enumerate(symptoms):
    # Use some heuristic to create a target variable for training (example: keyword matching)
    # Here we use a dummy target variable, in practice you need a properly labeled dataset
    y_symptom = np.random.randint(0, 5, size=X_text.shape[0])

    regressor.fit(X_text, y_symptom)
    symptom_ratings[:, i] = regressor.predict(X_text)

# Sum the predicted ratings to get the total score
total_scores = symptom_ratings.sum(axis=1)

# Define the depression levels
depression_levels = {
    (0, 5): 'No Depression',
    (6, 10): 'Normal but unhappy',
    (11, 25): 'Mild depression',
    (26, 50): 'Moderate depression',
    (51, 75): 'Severe depression',
    (76, 100): 'Extreme depression'
}

# Convert total scores to depression levels
def get_depression_level(score):
    for (low, high), level in depression_levels.items():
        if low <= score <= high:
            return level
    return 'Unknown'

depression_labels = np.array([get_depression_level(score) for score in total_scores])

# Convert depression levels to binary labels (example: Severe and Extreme depression as positive class)
binary_labels = np.array([1 if level in ['Severe depression', 'Extreme depression'] else 0 for level in depression_labels])

# Extract the 'PHQ8_Binary' label
labels = combined_data['PHQ8_Binary'].tolist()
y = np.array(labels)  # Ensure y is a numpy array

kf = KFold(n_splits=5)

f1_scores = []
sensitivities = []
specificities = []
precisions = []

for train_index, test_index in kf.split(X_text):
    X_train, X_test = X_text[train_index], X_text[test_index]
    y_train, y_test = y[train_index], y[test_index]

    # Handle class imbalance using SMOTE
    smote = SMOTE()
    X_train, y_train = smote.fit_resample(X_train, y_train)

    model = xgb.XGBClassifier(use_label_encoder=False, eval_metric='logloss')
    model.fit(X_train, y_train)

    y_pred = model.predict(X_test)

    f1 = f1_score(y_test, y_pred)
    sensitivity = recall_score(y_test, y_pred)
    specificity = specificity_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)

    f1_scores.append(f1)
    sensitivities.append(sensitivity)
    specificities.append(specificity)
    precisions.append(precision)

f1_mean = np.mean(f1_scores)
f1_std = np.std(f1_scores)
sensitivity_mean = np.mean(sensitivities)
sensitivity_std = np.std(sensitivities)
specificity_mean = np.mean(specificities)
specificity_std = np.std(specificities)
precision_mean = np.mean(precisions)
precision_std = np.std(precisions)

print(f"F1 Score: Mean = {f1_mean}, Std Dev = {f1_std}")
print(f"Sensitivity: Mean = {sensitivity_mean}, Std Dev = {sensitivity_std}")
print(f"Specificity: Mean = {specificity_mean}, Std Dev = {specificity_std}")
print(f"Precision: Mean = {precision_mean}, Std Dev = {precision_std}")


F1 Score: Mean = 0.22636363636363638, Std Dev = 0.1608448767607016
Sensitivity: Mean = 0.2204662004662005, Std Dev = 0.2315790176148754
Specificity: Mean = 0.9063376155059725, Std Dev = 0.06484470273385348
Precision: Mean = 0.4171428571428571, Std Dev = 0.32606497411856245


### **Claude 3**

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import StratifiedKFold
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, f1_score

# Assuming final_text is your list of lists containing text data
# final_text = [[utterance1, utterance2, ...], [utterance1, utterance2, ...], ...]

# Read the labels data
labels_df = pd.read_csv('train_split_Depression_AVEC2017.csv')

# Preprocess transcript data
def preprocess_transcript(utterances):
    # Convert all elements to strings and join
    return ' '.join(str(utterance) for utterance in utterances if utterance is not None)

# Extract features from transcripts
def extract_features(texts):
    vectorizer = TfidfVectorizer(max_features=1000, stop_words='english')
    features = vectorizer.fit_transform(texts)
    return pd.DataFrame(features.toarray(), columns=vectorizer.get_feature_names_out())

# Preprocess transcripts
processed_transcripts = [preprocess_transcript(transcript) for transcript in final_text]

# Extract features
features = extract_features(processed_transcripts)

# Prepare data for cross-validation
X = features
y = np.array(labels)  # Ensure y is a numpy array

# Initialize StratifiedKFold
skf = StratifiedKFold(n_splits=5, shuffle=True)

# Initialize lists to store performance metrics
specificities = []
sensitivities = []
f1_scores = []

# Perform 5-fold cross-validation
for fold, (train_index, val_index) in enumerate(skf.split(X, y), 1):
    X_train, X_val = X.iloc[train_index], X.iloc[val_index]
    y_train, y_val = y[train_index], y[val_index]

    # Scale the features
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_val_scaled = scaler.transform(X_val)

    # Apply SMOTE for oversampling
    smote = SMOTE(random_state=42)
    X_train_resampled, y_train_resampled = smote.fit_resample(X_train_scaled, y_train)

    # Train the model
    model = LogisticRegression(max_iter=1000, class_weight='balanced')
    model.fit(X_train_resampled, y_train_resampled)

    # Make predictions
    y_pred = model.predict(X_val_scaled)

    # Calculate metrics
    specificity, sensitivity, f1 = calculate_metrics(y_val, y_pred)

    # Store metrics
    specificities.append(specificity)
    sensitivities.append(sensitivity)
    f1_scores.append(f1)

    print(f"Fold {fold} - Specificity: {specificity:.4f}, Sensitivity: {sensitivity:.4f}, F1-score: {f1:.4f}")

# Print average metrics
print("\nAverage metrics:")
print(f"Specificity: {np.mean(specificities):.4f} (+/- {np.std(specificities):.4f})")
print(f"Sensitivity: {np.mean(sensitivities):.4f} (+/- {np.std(sensitivities):.4f})")
print(f"F1-score: {np.mean(f1_scores):.4f} (+/- {np.std(f1_scores):.4f})")

Fold 1 - Specificity: 0.7778, Sensitivity: 0.3636, F1-score: 0.3810
Fold 2 - Specificity: 0.7407, Sensitivity: 0.0909, F1-score: 0.1053
Fold 3 - Specificity: 0.7407, Sensitivity: 0.0909, F1-score: 0.1053
Fold 4 - Specificity: 0.7692, Sensitivity: 0.2500, F1-score: 0.2857
Fold 5 - Specificity: 0.8077, Sensitivity: 0.2727, F1-score: 0.3158

Average metrics:
Specificity: 0.7672 (+/- 0.0251)
Sensitivity: 0.2136 (+/- 0.1072)
F1-score: 0.2386 (+/- 0.1131)


In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import StratifiedKFold
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, f1_score
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE
import re

# ... (keep the previous import statements and function definitions)

# Define keywords for each item on the checklist
checklist_keywords = {
    'sad': ['sad', 'down', 'unhappy', 'blue', 'cry', 'tear'],
    'discouraged': ['discouraged', 'hopeless'],
    'self_esteem': ['worthless', 'inadequate', 'guilt', 'shame'],
    'criticizing': ['criticize', 'blame'],
    'decisions': ['difficult', 'decision'],
    'loss_interest': ['loss of interest', 'lonely', 'less time'],
    'motivation': ['unmotivated', 'avoid work'],
    'pleasure': ['no pleasure', 'no satisfaction'],
    'tired': ['tired', 'fatigue'],
    'sleep': ['sleep problem', 'insomnia'],
    'appetite': ['appetite', 'eating'],
    'sex': ['sex'],
    'health': ['health', 'worry'],
    'suicidal': ['suicide', 'end life', 'harm myself']
}

def estimate_ratings(text):
    ratings = []
    for category, keywords in checklist_keywords.items():
        count = sum(text.lower().count(keyword) for keyword in keywords)
        rating = min(count, 4)  # Cap at 4
        ratings.append(rating)
    return ratings

# Preprocess transcripts and estimate ratings
processed_transcripts = []
estimated_ratings = []
for transcript in final_text:
    processed_text = preprocess_transcript(transcript)
    processed_transcripts.append(processed_text)
    estimated_ratings.append(estimate_ratings(processed_text))

# Extract features
tfidf_features = extract_features(processed_transcripts)

# Combine TF-IDF features with estimated ratings
features = pd.concat([tfidf_features, pd.DataFrame(estimated_ratings, columns=checklist_keywords.keys())], axis=1)

# Prepare data for cross-validation
X = features
y = np.array(labels)  # Ensure y is a numpy array

# Initialize StratifiedKFold
skf = StratifiedKFold(n_splits=5, shuffle=True)

# Initialize lists to store performance metrics
specificities = []
sensitivities = []
f1_scores = []

# Perform 5-fold cross-validation
for fold, (train_index, val_index) in enumerate(skf.split(X, y), 1):
    X_train, X_val = X.iloc[train_index], X.iloc[val_index]
    y_train, y_val = y[train_index], y[val_index]

    # Scale the features
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_val_scaled = scaler.transform(X_val)

    # Apply SMOTE for oversampling
    smote = SMOTE(random_state=42)
    X_train_resampled, y_train_resampled = smote.fit_resample(X_train_scaled, y_train)

    # Train the model
    model = LogisticRegression(random_state=42, max_iter=1000, class_weight='balanced')
    model.fit(X_train_resampled, y_train_resampled)

    # Make predictions
    y_pred = model.predict(X_val_scaled)

    # Calculate metrics
    specificity, sensitivity, f1 = calculate_metrics(y_val, y_pred)

    # Store metrics
    specificities.append(specificity)
    sensitivities.append(sensitivity)
    f1_scores.append(f1)

    print(f"Fold {fold} - Specificity: {specificity:.4f}, Sensitivity: {sensitivity:.4f}, F1-score: {f1:.4f}")

# Print average metrics
print("\nAverage metrics:")
print(f"Specificity: {np.mean(specificities):.4f} (+/- {np.std(specificities):.4f})")
print(f"Sensitivity: {np.mean(sensitivities):.4f} (+/- {np.std(sensitivities):.4f})")
print(f"F1-score: {np.mean(f1_scores):.4f} (+/- {np.std(f1_scores):.4f})")

Fold 1 - Specificity: 0.7037, Sensitivity: 0.4545, F1-score: 0.4167
Fold 2 - Specificity: 0.8889, Sensitivity: 0.2727, F1-score: 0.3529
Fold 3 - Specificity: 0.8148, Sensitivity: 0.0909, F1-score: 0.1176
Fold 4 - Specificity: 0.8462, Sensitivity: 0.2500, F1-score: 0.3158
Fold 5 - Specificity: 0.6154, Sensitivity: 0.5455, F1-score: 0.4444

Average metrics:
Specificity: 0.7738 (+/- 0.1002)
Sensitivity: 0.3227 (+/- 0.1603)
F1-score: 0.3295 (+/- 0.1153)
